In [ ]:
#https://raw.githubusercontent.com/fintihlupik/NLP-sentimental/refs/heads/master/data/youtoxic_english_1000%20-%20youtoxic_english_1000.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

## Leer el Dataset

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/fintihlupik/NLP-sentimental/refs/heads/master/data/youtoxic_english_1000%20-%20youtoxic_english_1000.csv")

In [ ]:
df.head(10)

In [ ]:
print('La forma del dataset es', df.shape)
print('Rows duplicadas en el dataset :', df.duplicated().sum() )

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
%%capture
!pip install ydata-profiling
from ydata_profiling import ProfileReport
from IPython.display import HTML # import the HTML object

In [ ]:
profile_change = ProfileReport(df,title="Youtube Comments. Profiling Report", explorative=True,)
profile_change.to_file("output_change.html")
HTML(filename='output_change.html')

Hallazgos:

- No hay missing values.
- Las columnas IsHomophobic y IsRadicalism tienen valor constante de "False"
- El CommentId es un valor único.
- Los comentarios corresponden únicamente a 13 videos de youtube (13 VideoId)
- Las palabras más frecuentes son: and, the, a, to..
- El dataset está bastante balanceado en respecto a IsToxic
- El toxico está altamente relacionado con abusivo.
- IsSexist tiene solo un registro Positivo.
- Hay 997 comentarios diferentes, lo que sugiere que hay comentarios repetidos.



### Vacios y repetidos

In [ ]:
df = df.drop(columns=['CommentId', 'IsHomophobic', 'IsRadicalism'])

Borramos las columnas que no aportan valor

In [ ]:
print("Vacíos:", df[df['Text'].str.strip() == ''].shape[0])

No hay vacios

In [ ]:
num_duplicates = df['Text'].duplicated().sum()
print(f"Número de comentarios duplicados: {num_duplicates}")

In [ ]:
duplicates = df[df['Text'].duplicated(keep=False)]
print(duplicates)

In [ ]:
# Crear una columna temporal con texto en minúsculas para detectar duplicados
df['Text_lower'] = df['Text'].str.lower()

# Buscar duplicados según texto en minúsculas
duplicates_ignore_case = df[df['Text_lower'].duplicated(keep=False)]

print(duplicates_ignore_case)


### Distribución de clases (balanceo)

In [ ]:
sns.countplot(data=df, x='IsToxic')
plt.title("Distribución de etiquetas")
plt.show()

print(df['IsToxic'].value_counts(normalize=True))


In [ ]:
percentages = df['IsToxic'].value_counts(normalize=True) * 100
percentages = percentages.round(2)
print(percentages)

Como vimos anteriormente el dataset está bien balanceado.

## Longitud de comentarios

In [ ]:
df['comment_length'] = df['Text'].apply(lambda x: len(str(x).split()))
print(df['comment_length'].describe())

In [ ]:
plt.hist(df['comment_length'], bins=20)
plt.title("Distribución de longitud de los comentarios")
plt.xlabel("Número de palabras")
plt.ylabel("Frecuencia")
plt.show()

## Outliers

In [ ]:
sns.boxplot(x=df['comment_length'])
plt.title("Longitud de comentarios - Detección de outliers")
plt.xlabel("Número de palabras")
plt.show()

In [ ]:
Q1 = df['comment_length'].quantile(0.25)
Q3 = df['comment_length'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR # (el lower bound da -36, no hay outliers cortos)
upper_bound = Q3 + 1.5 * IQR # (84 - todos mayores a 84 son outliers)

# Comentarios fuera del rango aceptable
outliers = df[(df['comment_length'] < lower_bound) | (df['comment_length'] > upper_bound)]
print(f"Outliers detectados: {len(outliers)} comentarios")

# Porcentaje de outliers
print(f"{100 * len(outliers) / len(df):.2f}% del dataset son outliers de longitud")

### Longitud por clase

In [ ]:
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split()))
sns.boxplot(x='IsToxic', y='word_count', data=df)
plt.title("Distribución de longitud por clase")
plt.xlabel("¿Es tóxico?")
plt.ylabel("Número de palabras")
plt.show()

Vemos comentarios cortos

In [ ]:
short_comments = df[df['comment_length'] < 3][['Text', 'comment_length', 'IsToxic']].sort_values(by='comment_length')
print(short_comments)

Los comentarios cortos son válidos.

Vemos comentarios largos

In [ ]:
long_comments = df[df['comment_length'] > 150][['Text', 'comment_length', 'IsToxic']].sort_values(by='IsToxic')
print(long_comments)

Los outliers son comentarios válidos.

In [ ]:
# Definir los bins para los rangos de longitud
bins = [0, 9, 19, 39, 100, 150, 815]
labels = ['1-8', '9-18', '19-38', '39-99', '100-149', '150-815']

# Crear una nueva columna con el rango de longitud
df['length_range'] = pd.cut(df['comment_length'], bins=bins, labels=labels, right=False)

# Agrupar por rango y calcular porcentaje de tóxicos
toxicity_by_length = df.groupby('length_range')['IsToxic'].mean() * 100

# Crear gráfica de barras
plt.figure(figsize=(8,6))
sns.barplot(x=toxicity_by_length.index, y=toxicity_by_length.values, palette='viridis')

plt.title('Porcentaje de comentarios tóxicos por rango de longitud')
plt.xlabel('Rango de longitud (número de palabras)')
plt.ylabel('% de comentarios tóxicos')
plt.ylim(0, 100)
plt.show()


Como se puede ver la longitud del mensaje no parece ser un fuerte predictor de toxicidad del mensaje.

In [ ]:
# Convertimos las columnas booleanas a bool (por si hay categorías)
multi_tags = df.loc[:, 'IsAbusive':'IsReligiousHate'].astype(bool)

tag_totals = multi_tags.sum().sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=tag_totals.values, y=tag_totals.index, palette='viridis')
plt.title("Distribución de tipos de toxicidad")
plt.xlabel("Número de comentarios")
plt.ylabel("Etiqueta")
plt.show()
